### Imports

In [ ]:
import logging

import pandas as pd
from pandas import DataFrame as DF
import matplotlib.pyplot as plt

from core.plt_utils import plt_3d_df
from watea.watea_constants import *
from watea.processed_watea_ts import processed_ts_of, processed_ts_iterator
from watea.watea_fleet_info import fleet_info_df
from watea.energy_distribution import *

logging.basicConfig(level=logging.INFO)


### Toutes les distributions

In [ ]:
SUBJECT_ID = "hcz772"
SUBJECT_ID2 = "kdh372"


ts = processed_ts_of(SUBJECT_ID2)
energy_points = compute_charging_points(ts, SUBJECT_ID2) #.query("energy_added < 2000")

CHARGE_REGIME_CURRENT_INTERVALS = [(-11.5, -10), (-15, -12), (-23, -18), (-76, -64), (-100, -98)]


In [ ]:
# .query("energy_added > 300 & energy_added < 500 & sec_duration < 900")
plt_3d_df(energy_points, "voltage", "current", "energy_added", color="temp", colorscale="Bluered", size=4.5)

In [ ]:
for regime_current_interval in CHARGE_REGIME_CURRENT_INTERVALS:
    regime_energy_points = energy_points.query(f"mean_current > {regime_current_interval[0]} & mean_current < {regime_current_interval[1]}")
    max = regime_energy_points["mean_current"].quantile(0.9)
    regime_energy_points = regime_energy_points.query(f"mean_current < {max}")
    print(regime_current_interval)
    plt_3d_df(regime_energy_points, "mean_soc", "mean_current", "energy_added", color="charge_idx", colorscale="Rainbow", save_path=PATH_TO_FLEET_3D_DISTRIBUTION, size=4.5)

In [ ]:
fig, axs = plt.subplots(
    ncols=len(CHARGE_REGIME_CURRENT_INTERVALS),
    sharex=True,
    sharey=True,
    figsize=(15, 7)
)

for regime_current_interval, ax in zip(CHARGE_REGIME_CURRENT_INTERVALS, axs):
    regime_energy_points = energy_points.query(f"mean_current > {regime_current_interval[0]} & mean_current < {regime_current_interval[1]}")
    regime_energy_points.plot.scatter(
        x="mean_soc",
        y="energy_added",
        ax=ax,
        alpha=0.6,
        s=0.3,
        # c="mean_temp",
        # cmap="plasma"
    )

plt.show()


In [ ]:
# display(energy_points.dtypes)
agg_energy_points = (
    energy_points
    .loc[energy_points["mean_voltage"].notna()]
    # .loc[:, "energy_added"]
    # .drop(columns="duration")
    # .groupby(level=[0, 1, 3])
    # .rolling(window=10, min_periods=1, center=True, on="mean_voltage")
    # .median()
    # .droplevel([0, 1, 2])
    # .drop(columns=["charge_idx", "charge_id"])
)
# display(agg_energy_points.index)
fig, axs = plt.subplots(
    ncols=len(CHARGE_REGIME_CURRENT_INTERVALS),
    sharex=True,
    sharey=True,
    figsize=(15, 7)
)

for regime_current_interval, ax in zip(CHARGE_REGIME_CURRENT_INTERVALS, axs):
    regime_energy_points = agg_energy_points.query(f"mean_current > {regime_current_interval[0]} & mean_current < {regime_current_interval[1]}")
    regime_energy_points.plot.scatter(
        x="mean_soc",
        y="sec_duration",
        ax=ax,
        # alpha=0.6,
        s=0.6,
        # c="mean_current",
        # cmap="plasma"
    )

plt.show()


In [ ]:
fleet_energy_points = []
for id, ts in processed_ts_iterator("has_power_during_charge"):
    fleet_energy_points.append(compute_charging_points(ts, id))

fleet_energy_points = pd.concat(fleet_energy_points)

# plt_3d_df(fleet_energy_points.query("energy_added > 300 & sec_duration < 900 & temp < 35"), "charge_idx", "mean_current", "mean_soc", color="energy_added", colorscale="Bluered", size=4.5)

In [ ]:
fleet_energy_points_cleaned = fleet_energy_points.query("energy_added < 502")

In [ ]:
plt_3d_df(fleet_energy_points_cleaned, "voltage", "current", "energy_added", color="temp", colorscale="Bluered", size=2.5)
# plt_3d_df(fleet_energy_points_cleaned, "id_idx", "current", "energy_added", color="id_idx", colorscale="Rainbow", size=2.5)